# 如何强制工具调用行为

```{=mdx}

:::info 预备知识

本指南假定您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [LangChain 工具](/docs/concepts/tools)
- [如何使用模型调用工具](/docs/how_to/tool_calling)

:::

```

为了强制我们的语言模型选择特定工具，我们可以使用 `tool_choice` 参数来确保特定的行为。首先，我们定义我们的模型和工具：

In [1]:
import { tool } from '@langchain/core/tools';
import { z } from 'zod';

const add = tool((input) => {
    return `${input.a + input.b}`
}, {
    name: "add",
    description: "Adds a and b.",
    schema: z.object({
        a: z.number(),
        b: z.number(),
    })
})

const multiply = tool((input) => {
    return `${input.a * input.b}`
}, {
    name: "Multiply",
    description: "Multiplies a and b.",
    schema: z.object({
        a: z.number(),
        b: z.number(),
    })
})

const tools = [add, multiply]

In [2]:
import { ChatOpenAI } from '@langchain/openai';

const llm = new ChatOpenAI({
  model: "gpt-3.5-turbo",
})

例如，我们可以强制我们的工具通过使用以下代码来调用乘法工具：

In [3]:
const llmForcedToMultiply = llm.bindTools(tools, {
  tool_choice: "Multiply",
})
const multiplyResult = await llmForcedToMultiply.invoke("what is 2 + 4");
console.log(JSON.stringify(multiplyResult.tool_calls, null, 2));

[
  {
    "name": "Multiply",
    "args": {
      "a": 2,
      "b": 4
    },
    "type": "tool_call",
    "id": "call_d5isFbUkn17Wjr6yEtNz7dDF"
  }
]


即使我们传递给它一个不需要乘法运算的内容，它仍然会调用该工具！

我们也可以通过将 `"any"`（或者对于 OpenAI 模型，使用等效的 `"required"`）传递给 `tool_choice` 参数，强制我们的工具至少选择其中一个工具。

In [4]:
const llmForcedToUseTool = llm.bindTools(tools, {
  tool_choice: "any",
})
const anyToolResult = await llmForcedToUseTool.invoke("What day is today?");
console.log(JSON.stringify(anyToolResult.tool_calls, null, 2));

[
  {
    "name": "add",
    "args": {
      "a": 2,
      "b": 3
    },
    "type": "tool_call",
    "id": "call_La72g7Aj0XHG0pfPX6Dwg2vT"
  }
]
